This is a baseline for the HTCV Sem

In [5]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import shutil

from data_loader import load_data_entries
from constants import *

In [23]:
train_data_entries = load_data_entries(data_type="train",percentage_as_str="100")
print(len(train_data_entries))
print(train_data_entries)

2799
[('socal-fire_00000862_post_disaster.png', 3), ('hurricane-michael_00000504_post_disaster.png', 1), ('hurricane-florence_00000500_post_disaster.png', 3), ('midwest-flooding_00000264_post_disaster.png', 1), ('santa-rosa-wildfire_00000368_post_disaster.png', 1), ('hurricane-florence_00000218_post_disaster.png', 0), ('hurricane-florence_00000435_post_disaster.png', 1), ('hurricane-michael_00000322_post_disaster.png', 1), ('midwest-flooding_00000426_post_disaster.png', 1), ('hurricane-matthew_00000219_post_disaster.png', 3), ('santa-rosa-wildfire_00000351_post_disaster.png', 2), ('socal-fire_00000563_post_disaster.png', 0), ('socal-fire_00000367_post_disaster.png', 4), ('santa-rosa-wildfire_00000283_post_disaster.png', 2), ('hurricane-florence_00000022_post_disaster.png', 3), ('hurricane-florence_00000436_post_disaster.png', 3), ('socal-fire_00001178_post_disaster.png', 0), ('socal-fire_00000886_post_disaster.png', 2), ('socal-fire_00000073_post_disaster.png', 0), ('midwest-flooding_0

In [11]:
def visualize_data(idx, data_type, data):
    img_name = data[idx][0]
    img_path = os.path.join(DATASET_FOLDER_PATH,data_type,"images",img_name)
    img = Image.open(img_path)
    img.show()

In [12]:

# Define the ResNet model
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        self.resnet = models.resnet18(weights=None)
        self.resnet.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.resnet(x)
        return x

# Create an instance of the ResNet classifier
num_classes = 5  # Number of output classes
model = ResNetClassifier(num_classes)

In [17]:
# Load and preprocess your dataset using torchvision.transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the input image to match the expected size of ResNet (224x224)
    transforms.ToTensor()
])
# Calculate the mean and standard deviation of your dataset
path_to_dataset = os.path.join(DATASET_FOLDER_PATH,"train","images")
dataset = torchvision.datasets.ImageFolder(path_to_dataset, transform=transform)
data_point_names = np.array(train_data_entries)[:,0]
subset = [data for data in dataset if os.path.basename(data[0])in data_point_names]
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)

FileNotFoundError: Couldn't find any class folder in /Users/vinay/Uni/modules/HTCV/HTCV-Sem/src/../dataset/train/images.

In [ ]:
mean = 0.0
std = 0.0
total_samples = 0

for images, _ in loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    total_samples += batch_samples

mean /= total_samples
std /= total_samples

# Update the normalization transform using your dataset's mean and standard deviation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Example dataset loading using torchvision.datasets.ImageFolder
dataset = torchvision.datasets.ImageFolder(path_to_dataset, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

# Save the trained model
torch.save(model.state_dict(), 'resnet_classifier.pth')
